<a href="https://colab.research.google.com/github/anshupandey/MSA-analytics/blob/main/Mixed_Effects_Modesl_Labs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Module 9 — Mixed Effect Models (Labs 4–7)

**Dataset:** `Ocean_Hull_Insurance_datasetv2.csv`  
**Industry:** Ocean Hull Vessel Insurance

This notebook implements:

- **Lab 4:** Model selection & validation (AIC/BIC, LRT, Group-aware holdout/CV)
- **Lab 5:** Handling unbalanced & missing data
- **Lab 6:** Coefficient interpretation & ICC
- **Lab 7:** End-to-end application vs OLS

> Notes:  
> - Mixed models can be numerically sensitive. Start with simpler structures, then add complexity.  
> - Charts use Matplotlib (one plot per cell, default colors).


In [1]:

# ==== Setup ====
import pandas as pd, numpy as np, math
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pathlib import Path

%matplotlib inline

DATA_PATH = "https://raw.githubusercontent.com/anshupandey/MSA-analytics/refs/heads/main/datasets/Ocean_Hull_Insurance_datasetv2.csv"

df = pd.read_csv(DATA_PATH)

# Target for continuous modelling
df['log_claim'] = np.log1p(df['Claim_Amount'].astype(float))

# Categorical types
for c in ['Vessel_Type','Operating_Zone','Flag_State','Inspection_Status','Weather_Risk','Piracy_Risk']:
    df[c] = df[c].astype('category')

# Working frame (drop rows with missing in selected columns)
work = df[['log_claim','Claim_Amount','Vessel_Age','Premium','Sum_Insured',
           'Vessel_Type','Inspection_Status','Weather_Risk','Piracy_Risk','Operating_Zone']].dropna().copy()

print('Rows:', len(work), 'Zones:', work['Operating_Zone'].nunique())
work.head()


Rows: 300 Zones: 5


,log_claim,Claim_Amount,Vessel_Age,Premium,Sum_Insured,Vessel_Type,Inspection_Status,Weather_Risk,Piracy_Risk,Operating_Zone
0,0.0,0,19,159500,14050000,Container Ship,Overdue,Moderate,Moderate,Strait of Malacca
1,0.0,0,40,170000,13000000,Container Ship,Up-to-date,High,High,Strait of Malacca
2,0.0,0,26,193000,18700000,Tanker,Delayed,Low,Low,Indian Ocean
3,0.0,0,7,123500,11650000,Bulk Carrier,Up-to-date,Moderate,Low,South China Sea
4,0.0,0,18,109000,7100000,Offshore Support Vessel,Delayed,Low,Low,Mediterranean


In [2]:

# ==== Helpers ====

ols_formula = 'log_claim ~ Vessel_Age + Premium + Sum_Insured + C(Vessel_Type) + C(Inspection_Status) + C(Weather_Risk) + C(Piracy_Risk)'
me_formula  = ols_formula

def rmse(y, yhat): return float(np.sqrt(np.mean((y - yhat)**2)))
def mae(y, yhat):  return float(np.mean(np.abs(y - yhat)))

def fit_ols(data):
    return smf.ols(ols_formula, data=data).fit()

def fit_mixed_random_intercept(data, reml=True, maxiter=200):
    return sm.MixedLM.from_formula(me_formula, groups='Operating_Zone', re_formula='1', data=data).fit(reml=reml, maxiter=maxiter, method='lbfgs')

def fit_mixed_random_slope_age(data, reml=True, maxiter=200):
    return sm.MixedLM.from_formula(me_formula, groups='Operating_Zone', re_formula='1+Vessel_Age', data=data).fit(reml=reml, maxiter=maxiter, method='lbfgs')

def predict_mixed(fit_res, df_fold, formula=me_formula):
    # Uses fixed+random effects for known groups; for unseen groups, falls back to fixed effects.
    try:
        return fit_res.predict(df_fold)
    except Exception:
        import patsy
        y_, X = patsy.dmatrices(formula, data=df_fold, return_type='dataframe')
        betas = fit_res.fe_params.reindex(index=X.columns).fillna(0.0).values
        return np.dot(X.values, betas)

def group_holdout_metrics(train, test):
    m = {}
    ols_tr = fit_ols(train)
    m['OLS'] = (rmse(test['log_claim'], ols_tr.predict(test)), mae(test['log_claim'], ols_tr.predict(test)))
    me_ri_tr = fit_mixed_random_intercept(train)
    m['Mixed_RI'] = (rmse(test['log_claim'], predict_mixed(me_ri_tr, test)), mae(test['log_claim'], predict_mixed(me_ri_tr, test)))
    me_rs_tr = fit_mixed_random_slope_age(train)
    m['Mixed_RI_SlopeAge'] = (rmse(test['log_claim'], predict_mixed(me_rs_tr, test)), mae(test['log_claim'], predict_mixed(me_rs_tr, test)))
    return m, ols_tr, me_ri_tr, me_rs_tr

def icc_from_fit(fit):
    try:
        re_var = float(np.diag(fit.cov_re)[0])
        resid_var = float(fit.scale)
        return re_var/(re_var+resid_var)
    except Exception:
        return np.nan



---
## Lab 4 — Model Selection & Validation

1. Fit **OLS**, **Mixed RI**, **Mixed RI + Age slope** on the full dataset.  
2. Compare **AIC/BIC** and (if available) **LRT** for RS vs RI.  
3. Validate with a **group-aware holdout** (20% Operating_Zone).

> If optimization struggles, start with **Mixed RI** (wider stability), then add the slope.


In [3]:

# 4.1 Full-sample fits
ols = fit_ols(work)
print(ols.summary())

me_ri = fit_mixed_random_intercept(work)
print(me_ri.summary())

# Random slope; if it fails, comment this out or try smaller model
try:
    me_rs = fit_mixed_random_slope_age(work)
    print(me_rs.summary())
except Exception as e:
    me_rs = None
    print('Random-slope model failed:', e)


                            OLS Regression Results                            
Dep. Variable:              log_claim   R-squared:                       0.710
Model:                            OLS   Adj. R-squared:                  0.700
Method:                 Least Squares   F-statistic:                     70.62
Date:                Thu, 14 Aug 2025   Prob (F-statistic):           1.26e-71
Time:                        14:03:41   Log-Likelihood:                -783.30
No. Observations:                 300   AIC:                             1589.
Df Residuals:                     289   BIC:                             1629.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------

/usr/local/lib/python3.11/dist-packages/statsmodels/regression/mixed_linear_model.py:1634: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/statsmodels/regression/mixed_linear_model.py:2054: UserWarning: The random effects covariance matrix is singular.
  warnings.warn(_warn_cov_sing)
/usr/local/lib/python3.11/dist-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.11/dist-packages/statsmodels/regression/mixed_linear_model.py:2245: UserWarning: The random effects covariance matrix is singular.
  warnings.warn(_warn_cov_sing)
/usr/local/lib/python3.11/dist-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.11/d

Random-slope model failed: Singular matrix


In [4]:

# 4.2 Information criteria & LRT
def safe_num(x):
    try: return float(x)
    except: return float('nan')

results = {
    'OLS_AIC': safe_num(ols.aic), 'OLS_BIC': safe_num(ols.bic),
    'ME_RI_AIC': safe_num(me_ri.aic), 'ME_RI_BIC': safe_num(me_ri.bic),
    'ME_RS_AIC': safe_num(me_rs.aic) if me_rs is not None else float('nan'),
    'ME_RS_BIC': safe_num(me_rs.bic) if me_rs is not None else float('nan'),
}
print(results)

# Likelihood-ratio test (RS vs RI) if RS was fitted
if me_rs is not None:
    try:
        stat, pval, df_ = me_rs.compare_lr_test(me_ri)
        print('LRT (RS vs RI): stat=%.3f, p=%.4f, df=%d' % (stat, pval, df_))
    except Exception as e:
        print('LRT not available:', e)


{'OLS_AIC': 1588.5910600256461, 'OLS_BIC': 1629.3326672468643, 'ME_RI_AIC': nan, 'ME_RI_BIC': nan, 'ME_RS_AIC': nan, 'ME_RS_BIC': nan}


In [5]:

# 4.3 Group-aware holdout (20% zones)
rng = np.random.default_rng(123)
zones = work['Operating_Zone'].cat.categories.tolist()
rng.shuffle(zones)
n_test = max(1, int(0.2*len(zones)))
test_z = set(zones[:n_test])
train = work[~work['Operating_Zone'].isin(test_z)].copy()
test  = work[ work['Operating_Zone'].isin(test_z)].copy()

metrics, ols_tr, me_ri_tr, me_rs_tr = group_holdout_metrics(train, test)
metrics


/usr/local/lib/python3.11/dist-packages/statsmodels/regression/mixed_linear_model.py:1634: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/statsmodels/regression/mixed_linear_model.py:2054: UserWarning: The random effects covariance matrix is singular.
  warnings.warn(_warn_cov_sing)
/usr/local/lib/python3.11/dist-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.11/dist-packages/statsmodels/regression/mixed_linear_model.py:2245: UserWarning: The random effects covariance matrix is singular.
  warnings.warn(_warn_cov_sing)
/usr/local/lib/python3.11/dist-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


LinAlgError: Singular matrix


> **Interpretation checklist:**  
> - Which model has the lowest **AIC/BIC**?  
> - Does the **LRT** favor adding a random slope?  
> - In holdout, which model has the lowest **RMSE/MAE**?  
> - If RI+Slope overfits or fails to converge, prefer RI.



---
## Lab 5 — Handling Unbalanced & Missing Data

We will induce group imbalance and missing values, then compare two strategies:
- **Listwise deletion**
- **Simple imputation** (median for numerics)

> Mixed models handle **unbalanced groups** by design, but **missing data** must be handled explicitly (assume MAR if using deletion/imputation).


In [ ]:

# 5.1 Create imbalance & missingness
rng = np.random.default_rng(7)
tmp = work.copy()

# Down-sample 70% of rows from 30% of zones
zones = tmp['Operating_Zone'].cat.categories.tolist()
drop_z = set(rng.choice(zones, size=max(1, int(0.3*len(zones))), replace=False))
to_drop = tmp[tmp['Operating_Zone'].isin(drop_z)].sample(frac=0.7, random_state=7).index
tmp_imbal = tmp.drop(index=to_drop).copy()

# Inject ~10% missing in key numerics
for col in ['Vessel_Age','Premium','Sum_Insured']:
    m = rng.random(len(tmp_imbal)) < 0.10
    tmp_imbal.loc[m, col] = np.nan

print('Post-imbalance rows:', len(tmp_imbal))


In [ ]:

# 5.2 Strategy A: Listwise deletion
a_del = tmp_imbal.dropna().copy()
fit_del = fit_mixed_random_slope_age(a_del)
print('AIC (deletion):', float(fit_del.aic))


In [ ]:

# 5.3 Strategy B: Simple imputation
b_imp = tmp_imbal.copy()
for col in ['Vessel_Age','Premium','Sum_Insured']:
    b_imp[col] = b_imp[col].fillna(b_imp[col].median())
fit_imp = fit_mixed_random_slope_age(b_imp)
print('AIC (imputation):', float(fit_imp.aic))



> **Interpretation checklist:**  
> - Which approach yields lower **AIC**?  
> - Do coefficients and random-effect variances remain stable?  
> - Document your assumption about the missingness mechanism (MCAR/MAR/MNAR).



---
## Lab 6 — Coefficient Interpretation & ICC

Using your **selected model** (e.g., Mixed RI or Mixed RI+Slope):
- Convert coefficients on `log_claim` to **multipliers** on the claim scale via `exp(beta)`.
- Compute an **ICC** to quantify between-zone variation.


In [ ]:

# Select a fitted model from Lab 4 (change as needed)
best_fit = me_ri if 'me_ri' in globals() else fit_mixed_random_intercept(work)

# Multipliers on claim scale
params = best_fit.fe_params
def mult(beta): return float(np.exp(beta))

for var in ['Vessel_Age','Premium','Sum_Insured']:
    if var in params.index:
        print(f'{var}: beta={params[var]:.6f} -> multiplier ≈ {mult(params[var]):.6f}')

# ICC
def icc_from_fit(fit):
    try:
        re_var = float(np.diag(fit.cov_re)[0]); resid_var = float(fit.scale)
        return re_var/(re_var+resid_var)
    except Exception: return np.nan

print('ICC (Operating_Zone) ≈', icc_from_fit(best_fit))



> **Interpretation checklist:**  
> - Translate multipliers into **business language** (e.g., “each extra year of vessel age changes expected claim by X%”).  
> - How large is the ICC? If high, zone-level factors significantly impact risk and pricing.



---
## Lab 7 — End-to-End Application & Comparison with OLS

1. Fit baseline **OLS** and your **selected Mixed model**.  
2. Compare **AIC** and **holdout** error.  
3. Produce **diagnostic plots** (fitted vs actual; residuals).  
4. Summarize business-impact insights.


In [ ]:

# 7.1 Fit baseline and selected models
ols_full = fit_ols(work)
me_full  = fit_mixed_random_slope_age(work)  # change to RI if RS fails

print('AIC (OLS):', float(ols_full.aic), 'AIC (Mixed):', float(me_full.aic))

# 7.2 Fitted vs Actual
plt.figure()
plt.scatter(ols_full.fittedvalues, work['log_claim'], alpha=0.6)
plt.title('OLS: Fitted vs Actual (log_claim)')
plt.xlabel('Fitted')
plt.ylabel('Actual')
plt.show()

plt.figure()
plt.scatter(me_full.fittedvalues, work['log_claim'], alpha=0.6)
plt.title('Mixed: Fitted vs Actual (log_claim)')
plt.xlabel('Fitted')
plt.ylabel('Actual')
plt.show()

# 7.3 Residuals vs Fitted (Mixed)
plt.figure()
plt.scatter(me_full.fittedvalues, me_full.resid, alpha=0.6)
plt.axhline(0, linestyle='--')
plt.title('Mixed: Residuals vs Fitted')
plt.xlabel('Fitted')
plt.ylabel('Residuals')
plt.show()



### 1-Page Insight Summary (complete after running)
- **Selected structure:** (record RI or RI+Slope and why)  
- **Between-zone heterogeneity:** ICC and which zones show largest random intercepts  
- **Key drivers:** Convert top fixed effects to %.  
- **Why mixed > OLS:** Better generalization across zones; avoids biased estimates in sparse zones.  
- **Underwriting actions:** Zone-based pricing adjustments; review inspection schedules; tie to weather/piracy risk.
